# Intro to test-driven development

Test driven development is a technique where you write the tests for your code
before you write your code.  This technique when applied in the right
circumstances has a very well proven industry track record in terms of
improving quality of code and reducing overall defects count and as such is an
important technique to have in your repertoire.

## Why would we want to write tests first?

One of the most jarring things about test driven development for people that
are not used to it is the act of writing out the test first.

This mindset is not something that tends to come naturally
at first (it didn't for me initially) because most people are used to writing
the code first and get an introduction to the software engineering practice of
testing later on in their journey of learning to develop software.

The main reason that firms will go to the effort and expense of hiring a
dedicated team for testing or QA that is completely separate to the team making
the product is because it's actually quite hard to test things that you have
made. The mindset of creation is not the same one that really helps for testing
of trying to pull everything apart.

To effectively test a code base involves a mindset of mercilessly pulling
everything apart, looking for any edge cases that lead to bugs, looking for
that one in a million bug that rarely comes up.

However it goes a little beyond this, when we are testing out own code some
biases come into play:

### Confirmation Bias

> Definition: The tendency to search for,
interpret, focus on and remember information in a way that confirms one’s
preconceptions. 

It can be hard to search for ways in which your program does not work, even if
that's what you want to do this bias can creep in and make it hard for you to
see the ways in which you can make your code break, even if that's what you are
trying to do!

If you write the tests first: You don't have an implementation yet that will
break, so it's easier to write tests that will break your future code without
running into this cognitive bias. Because the code doesn't exist yet you don't
have a chance to get swayed by your existing preconceptions.

### Testing the implementation details

When you are writing some code you are very intimately aware of how it will
generate the results you want.

When we have effective tests we are really just wanting to test the
interface and properties of how the code transforms inputs to outputs. *How*
exactly the code does this is not what we want to test, the implementation of
the code may be changed later and a sizable part of good code architecture is
enabling changes in implementation to not break existing code that uses it.
Knowing more about the code implementation can in some cases actually make it
harder to write a good test!

For example lets say you have a situation where you have a function where you
are trying to multiply two numbers together.

Let's say the way it does it is to call a web API for a calculation service that calculates the numbers on a server and returns them back to you.
You might decide to test that the function worked by logging in to the service
to see if the API call was successful and see if the return value matched what you expected.

While this may test your function adequately it's testing it via the incidental way
in which it is implemented, what if this changes later? What if we want to move to a
different service or different approach? If we have our tests written this way
we can have our tests end up failing even when the function keeps working perfectly!

If you write the tests first: there are no implementation details yet. So it's
much harder to fall into the trap of testing the implementation instead of the
interface.

### The tests can't be skipped

For a variety of reasons pressure might come up to skip tests entirely.
Sometimes this is appropriate but very frequently it is not. The internal
quality of a project is not something that can be traded off for speed
(https://martinfowler.com/bliki/TradableQualityHypothesis.html) because
internal quality of your codebase is the enabler of speed of development of
features.

By writing the tests first people will not skip out on testing, this can be a
good practice for junior engineers who haven't built up the judgment to know
where tests can be skipped.

Also for organizational reasons it might be much more preferable to write the
tests first if there is a lack of procedural effectiveness that can get in the
way of the software development process. For example if there is a large amount
of churn of personnel on a particular project there's a substantial risk that
things such as tests and documentation will never get written if it's not done
incrementally along with the code. Coming back up to speed on these things is a
substantial productivity killer because the lower the gap in time and gap in
communications there is between writing the code and tests the easier it is to
do.

## How to write the tests first: Red, Green, Refactor

There are many ways that people go about writing tests for their code.
Sometimes people write tests after they have some functionality because they
know something that the code should do and they wish to ensure they do it.
Another approach is to write the tests before you have written the code, there
is a workflow that is highly effective for this called "red, green then
refactor".

This technique comprises the following 3 steps:

1. Red: Write a test that will both test your code appropriately and will fail when you run it
2. Green: Implement the minimal amount of code to make the test you just wrote pass
3. Refactor: Improve the code from the second step

The steps in the process here are quite important and it's important that they
are 3 distinct steps.  We will break this down step by step with an example where
are making a function that will multiply two numbers together.

## Step 1: Red

The first step when considering a test is what you expect the output to be for
some given input.

This will help you clarify what the interface of your functionality is and what
the requirements are.

We will use pytest for these examples, if you don't have pytest installed yet you can install it with:

In [ ]:
%%bash
pip install pytest

The first step is to make a test that just tries to call the functionality.
In this case we want to see if we can to import the module/function.

In [ ]:
# test_multiply.py
def test_multiply_function():
    from tdd_example import multiply_two

Because the function does not exist yet this will fail.

In [ ]:
%%bash
(master)$ python -m pytest .
========================= test session starts =========================
platform linux -- Python 3.7.2, pytest-3.10.1, py-1.6.0, pluggy-0.7.1
rootdir: /home/janis/TDD-example, inifile:
plugins: remotedata-0.3.0, openfiles-0.3.0, doctestplus-0.1.3, arraydiff-0.2
collected 1 item                                                      

test_multiply.py F                                              [100%]

============================== FAILURES ===============================
_______________________ test_multiply_function ________________________

    def test_multiply_function():
>       from tdd_example import multiply_two
E       ModuleNotFoundError: No module named 'tdd_example'

test_multiply.py:2: ModuleNotFoundError
====================== 1 failed in 0.03 seconds =======================

This is good, as it shows that our test is failing.

Let's make a stub function now but deliberately not try to do anything with it:

In [ ]:
# tdd_example.py
def multiply_two(first, second):
    """Multiply the two arguments together"""
    raise NotImplementedError

Now when we run the test again we get the following:

In [ ]:
%%bash
(master)$ python -m pytest .
========================= test session starts =========================
platform linux -- Python 3.7.2, pytest-3.10.1, py-1.6.0, pluggy-0.7.1
rootdir: /home/janis/TDD-example, inifile:
plugins: remotedata-0.3.0, openfiles-0.3.0, doctestplus-0.1.3, arraydiff-0.2
collected 1 item                                                      

test_multiply.py .                                              [100%]

====================== 1 passed in 0.02 seconds =======================

Now this does what the test case needs and passes, but there's the issue that 
the function doesn't actually do anything useful yet!
Despite this we are making progress, we know we can import the function, we just have to implement it.

At this point it's important we think about the function requirements and make sure
we figure create some tests that will indicate the function works AND that make the tests go red again.

In [ ]:
# test_multiply.py
def test_multiply_function():
    from tdd_example import multiply_two

    assert multiply_two(1, 4) == 4
    assert multiply_two(0, 3) == 0

Now when we run this:

In [ ]:
%%bash
========================= test session starts =========================
platform linux -- Python 3.7.2, pytest-3.10.1, py-1.6.0, pluggy-0.7.1
rootdir: /home/janis/TDD-example, inifile:
plugins: remotedata-0.3.0, openfiles-0.3.0, doctestplus-0.1.3, arraydiff-0.2
collected 1 item                                                      

test_multiply.py F                                              [100%]

============================== FAILURES ===============================
_______________________ test_multiply_function ________________________

    def test_multiply_function():
        from tdd_example import multiply_two
    
>       assert multiply_two(1, 4) == 4

test_multiply.py:4: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

first = 1, second = 4

    def multiply_two(first, second):
        """Multiply the two arguments together"""
>       raise NotImplementedError
E       NotImplementedError

tdd_example.py:3: NotImplementedError
====================== 1 failed in 0.07 seconds =======================

Now we get a failure from the test we just added to, this is what we want since
we haven't implemented the multiplication functionality just yet.


### Why it's important that the test fails at the red step

You'll have noticed before that there's some situations where the test passes
despite the functionality not being entirely there, you want to make sure in any
cases like this you change your tests to make sure that missing functionality fails.
If your tests pass without the code being implemented it means that your test cases
don't actually have any power, you don't want to have to wonder if the test passes
because of some incidental factor.

So if any test passes at the red step before there's implementation, this is an issue.
You'll want to build a habit of fixing up any bad test cases as soon as possible.
Bad test suites in the worst case this can actually give you false confidence
that your program is working when in fact it is not.

Having the test fail initially is important because the feedback you get when you implement in the next step (green) will then be much more meaningful.

## Step 2: Green

This is the step where we write the code to make our functionality work. In
some workflows people are used to doing this step first. The fact that this
step is *not* first is the defining characteristic of Test Driven Development,
the test is the initial driver of the development in this mindset, not the
other way around.

In this step it is important to not be perfectionistic, you want to do the
absolute minimum amount of coding to make your test cases pass. There's 2
reasons for this:

1. Firstly you want to make sure that you can meet your requirements as stated as fast as possible
2. When you improve your code it is helpful to have a test case that will show you if
your improvements have actually broken the test case.

In this case the implementation to get the tests green is fairly straightforward:

In [ ]:
# test_multiply.py
def multiply_two(first, second):
    """Multiply the two arguments together"""
    return first * second

Let's run the test case:

In [ ]:
%%bash
(master)$ python -m pytest
========================= test session starts =========================
platform linux -- Python 3.7.2, pytest-3.10.1, py-1.6.0, pluggy-0.7.1
rootdir: /home/janis/TDD-example, inifile:
plugins: remotedata-0.3.0, openfiles-0.3.0, doctestplus-0.1.3, arraydiff-0.2
collected 1 item                                                      

test_multiply.py .                                              [100%]

====================== 1 passed in 0.02 seconds =======================

Now we have a passing test!

### Less code is better

In general, by doing the minimal amount necessary to make the test pass, you
give yourself a prototyping opportunity. This step can gives you
insight into the best way of doing things. By making a prototype of sorts you
get to see not only how to make an improved version but whether you need to
improve it at all.

Sometimes what
you get in step 2 ends up being good enough and you can just move on to more
productive tasks. But when step 2 is not good enough it's much easier to use
the knowledge you gained in the actual act of doing it to improve your code
than it is to hypothesize about it beforehand.

It turns out that hypothesizing beforehand isn't always even possible because
you don't know what you will learn in the process ahead of time.

(There is a great book by Kenneth O'Stanley called "Why greatness cannot be
planned: the myth of the objective function".  This explains the mechanism by
which this iterative approach to learning allows you to produce things that are
not possible if you attempted to work at an objective from the start. This is
because you get to learn along the way, including things you didn't anticipate
ahead of time.)

## Step 3: Refactor

This is the step where you can improve the code in various ways.

This is a great time to reduce the amount of technical debt that your
code has, you have a test case or a few test cases that will inform you if you
make a mistake and this will give you instant feedback on latest changes that
can uncover bugs extremely efficiently.

Let's say someone tries using the function with a string:

In [ ]:
>>> from tdd_example import multiply_two
>>> multiply_two("abc", 2)
'abcabc'
>>> multiply_two("4", 2)
'44'

Perhaps we don't want our function to allow strings as input for some reason.
In this situation we may decide to change the function to raise a `TypeError`
if a string is provided.

In this case we would add a new test case that catches this issue:

In [ ]:
# test_multiply.py
def test_multiply_function():
    from tdd_example import multiply_two

    assert multiply_two(1, 4) == 4
    assert multiply_two(0, 3) == 0

def test_multiply_function_no_strings():
    """Test that attempting to use the multiply function with
    a string as an argument raises TypeError"""
    from tdd_example import multiply_two

    import pytest
    with pytest.raises(TypeError):
        multiply_two("abc", 2)

Now when we run this:

In [ ]:
%%bash
(master)$ python -m pytest
========================= test session starts =========================
platform linux -- Python 3.7.2, pytest-3.10.1, py-1.6.0, pluggy-0.7.1
rootdir: /home/janis/TDD-example, inifile:
plugins: remotedata-0.3.0, openfiles-0.3.0, doctestplus-0.1.3, arraydiff-0.2
collected 2 items                                                     

test_multiply.py .F                                             [100%]

============================== FAILURES ===============================
__________________ test_multiply_function_no_strings __________________

    def test_multiply_function_no_strings():
        """Test that attempting to use the multiply function with
        a string as an argument raises TypeError"""
        from tdd_example import multiply_two
    
        import pytest
        with pytest.raises(TypeError):
>           multiply_two("abc", 2)
E           Failed: DID NOT RAISE <class 'TypeError'>

test_multiply.py:14: Failed
================= 1 failed, 1 passed in 0.06 seconds ==================

We see that we have our original test still passing, and we will want to make sure
we keep it passing. However the newer test is not passing yet which is what we want.

We can implement this functionality now in the function:

In [ ]:
def multiply_two(first, second):
    """Multiply the two arguments together"""
    if isinstance(first, str) or isinstance(second, str):
        raise TypeError("Strings are not allowed")
    return first * second

Now when we run the tests:

In [ ]:
%%bash
(master)$ python -m pytest
========================= test session starts =========================
platform linux -- Python 3.7.2, pytest-3.10.1, py-1.6.0, pluggy-0.7.1
rootdir: /home/janis/TDD-example, inifile:
plugins: remotedata-0.3.0, openfiles-0.3.0, doctestplus-0.1.3, arraydiff-0.2
collected 2 items                                                     

test_multiply.py ..                                             [100%]

====================== 2 passed in 0.02 seconds =======================

The important part of the refactor step is to make improvements to the code that do not break the tests that were passing before.
When you get used to working in this fashion where you make tests first then functionality second it can help you feel more confident that your newer changes won't break older code.

### TDD exercise

We will run through writing some new functionality in a TDD manner.